## Tool Use

*(Coding along with deeplearning.ai's online course [Building toward Computer Use with Anthropic - Learn how an AI Assistant is built to use and accomplish tasks on computers](https://learn.deeplearning.ai/courses/building-toward-computer-use-with-anthropic/lesson/1/introduction) taught by Colt Steele)*

In [1]:
# https://learn.deeplearning.ai/courses/building-toward-computer-use-with-anthropic/lesson/3/working-with-the-api
from anthropic import Anthropic
import pandas as pd

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME="claude-3-5-sonnet-20241022"

Don't be a fool and sent your api key to github


#### __What Can You Do With Tool Use?__ 

- Retrieve data
- Query databases
- Interact with APIs
- Execute code
- Search the web
- Control a computer

#### __Tool Use Flow__

1. Claude wants to call a tool like inventory_lookup()
2. We run the underlying tool, querying the database
3. Database returns matching data
4. We provide Claude with the result

<img src="../assets/images/tool_use.png" width="70%" />

#### __Our Fake Database__

In [23]:
class FakeDatabase:
    def __init__(self):
        self.customers = [
            {"id": "1213210", "name": "John Doe", "email": "john@gmail.com", "phone": "123-456-7890", "username": "johndoe"},
            {"id": "2837622", "name": "Priya Patel", "email": "priya@candy.com", "phone": "987-654-3210", "username": "priya123"},
            {"id": "3924156", "name": "Liam Nguyen", "email": "lnguyen@yahoo.com", "phone": "555-123-4567", "username": "liamn"},
            {"id": "4782901", "name": "Aaliyah Davis", "email": "aaliyahd@hotmail.com", "phone": "111-222-3333", "username": "adavis"},
            {"id": "5190753", "name": "Hiroshi Nakamura", "email": "hiroshi@gmail.com", "phone": "444-555-6666", "username": "hiroshin"},
            {"id": "6824095", "name": "Fatima Ahmed", "email": "fatimaa@outlook.com", "phone": "777-888-9999", "username": "fatimaahmed"},
            {"id": "7135680", "name": "Alejandro Rodriguez", "email": "arodriguez@protonmail.com", "phone": "222-333-4444", "username": "alexr"},
            {"id": "8259147", "name": "Megan Anderson", "email": "megana@gmail.com", "phone": "666-777-8888", "username": "manderson"},
            {"id": "9603481", "name": "Kwame Osei", "email": "kwameo@yahoo.com", "phone": "999-000-1111", "username": "kwameo"},
            {"id": "1057426", "name": "Mei Lin", "email": "meilin@gmail.com", "phone": "333-444-5555", "username": "mlin"}
        ]

        self.orders = [
            {"id": "24601", "customer_id": "1213210", "product": "Wireless Headphones", "quantity": 1, "price": 79.99, "status": "Shipped"},
            {"id": "13579", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 2, "price": 19.99, "status": "Processing"},
            {"id": "97531", "customer_id": "2837622", "product": "Bluetooth Speaker", "quantity": 1, "price": "49.99", "status": "Shipped"}, 
            {"id": "86420", "customer_id": "3924156", "product": "Fitness Tracker", "quantity": 1, "price": 129.99, "status": "Delivered"},
            {"id": "54321", "customer_id": "4782901", "product": "Laptop Sleeve", "quantity": 3, "price": 24.99, "status": "Shipped"},
            {"id": "19283", "customer_id": "5190753", "product": "Wireless Mouse", "quantity": 1, "price": 34.99, "status": "Processing"},
            {"id": "74651", "customer_id": "6824095", "product": "Gaming Keyboard", "quantity": 1, "price": 89.99, "status": "Delivered"},
            {"id": "30298", "customer_id": "7135680", "product": "Portable Charger", "quantity": 2, "price": 29.99, "status": "Shipped"},
            {"id": "47652", "customer_id": "8259147", "product": "Smartwatch", "quantity": 1, "price": 199.99, "status": "Processing"},
            {"id": "61984", "customer_id": "9603481", "product": "Noise-Cancelling Headphones", "quantity": 1, "price": 149.99, "status": "Shipped"},
            {"id": "58243", "customer_id": "1057426", "product": "Wireless Earbuds", "quantity": 2, "price": 99.99, "status": "Delivered"},
            {"id": "90357", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 1, "price": 19.99, "status": "Shipped"},
            {"id": "28164", "customer_id": "2837622", "product": "Wireless Headphones", "quantity": 2, "price": 79.99, "status": "Processing"}
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")
        
        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None
    
    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"

In [24]:
# instantiating the fake database
db = FakeDatabase()

In [25]:
# getting customer in order to find his orders
db.get_user("email","john@gmail.com")

{'id': '1213210',
 'name': 'John Doe',
 'email': 'john@gmail.com',
 'phone': '123-456-7890',
 'username': 'johndoe'}

In [6]:
# getting our customer's order
db.get_customer_orders('1213210')

[{'id': '24601',
  'customer_id': '1213210',
  'product': 'Wireless Headphones',
  'quantity': 1,
  'price': 79.99,
  'status': 'Shipped'},
 {'id': '13579',
  'customer_id': '1213210',
  'product': 'Smartphone Case',
  'quantity': 2,
  'price': 19.99,
  'status': 'Processing'},
 {'id': '90357',
  'customer_id': '1213210',
  'product': 'Smartphone Case',
  'quantity': 1,
  'price': 19.99,
  'status': 'Shipped'}]

In [7]:
# cancelling the order
db.cancel_order('13579')

'Cancelled the order'

In [8]:
db.get_order_by_id('13579')

{'id': '13579',
 'customer_id': '1213210',
 'product': 'Smartphone Case',
 'quantity': 2,
 'price': 19.99,
 'status': 'Cancelled'}

### Defining Tool Schemas

In [9]:
# hypothetical tool definition for the get_user method
{
    "name": "get_user",
    "description": "Looks up a user by email, phone, or username.",
    "input_schema": {
        "type": "object",
        "properties": {
            "key": {
                "type": "string",
                "enum": ["email", "phone", "username"],
                "description": "The attribute to search for a user by (email, phone, or username)."
            },
            "value": {
                "type": "string",
                "description": "The value to match for the specified attribute."
            }
        },
        "required": ["key", "value"]
    }
}

{'name': 'get_user',
 'description': 'Looks up a user by email, phone, or username.',
 'input_schema': {'type': 'object',
  'properties': {'key': {'type': 'string',
    'enum': ['email', 'phone', 'username'],
    'description': 'The attribute to search for a user by (email, phone, or username).'},
   'value': {'type': 'string',
    'description': 'The value to match for the specified attribute.'}},
  'required': ['key', 'value']}}

### All Four Tools We're Using

In [11]:
tools = [
    {
        "name": "get_user",
        "description": "Looks up a user by email, phone, or username.",
        "input_schema": {
            "type": "object",
            "properties": {
                "key": {
                    "type": "string",
                    "enum": ["email", "phone", "username"],
                    "description": "The attribute to search for a user by (email, phone, or username)."
                },
                "value": {
                    "type": "string",
                    "description": "The value to match for the specified attribute."
                }
            },
            "required": ["key", "value"]
        }
    },
    {
        "name": "get_order_by_id",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "get_customer_orders",
        "description": "Retrieves the list of orders belonging to a user based on a user's customer id.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The customer_id belonging to the user"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on a provided order_id.  Only orders that are 'processing' can be cancelled",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order_id pertaining to a particular order"
                }
            },
            "required": ["order_id"]
        }
    }
]

### Giving Claude Tools

In [15]:
messages = [
    {
        "role": "user",
        "content": "Show me my orders.  My username is priya123"
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    # passing tools as additional parameters
    tools=tools,
    messages=messages
)

In [16]:
print(response.content) # inludes ToolUseBlock

[TextBlock(citations=None, text="I'll help you look up your orders. Let me first get your user information using your username, and then I'll retrieve your orders.", type='text'), ToolUseBlock(id='toolu_013DTzSnx3xo4MHDUg3rjAFi', input={'key': 'username', 'value': 'priya123'}, name='get_user', type='tool_use')]


In [17]:
print(response.content[-1])

ToolUseBlock(id='toolu_013DTzSnx3xo4MHDUg3rjAFi', input={'key': 'username', 'value': 'priya123'}, name='get_user', type='tool_use')


In [18]:
# the model tells us in the ToolUseBlock what it wants to do
# it's up to us now to actually execute this tool call
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_user":
        return db.get_user(tool_input["key"], tool_input["value"])
    elif tool_name == "get_order_by_id":
        return db.get_order_by_id(tool_input["order_id"])
    elif tool_name == "get_customer_orders":
        return db.get_customer_orders(tool_input["customer_id"])
    elif tool_name == "cancel_order":
        return db.cancel_order(tool_input["order_id"])

In [19]:
tool_use = response.content[-1] 
tool_name = tool_use.name
tool_input = tool_use.input

In [20]:
tool_result = process_tool_call(tool_name, tool_input) # result the model expects

In [21]:
# sending the result back to the model
# structure of the follow up message we need to send
{
    "role": "user",
    "content": [
        {
            "type": "tool_result", # important: content type is tool_result
            "tool_use_id": tool_use.id, # the id from the ToolUseBlock from above
            "content": str(tool_result), # here we need to send the actual content
        }
    ],
}

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_013DTzSnx3xo4MHDUg3rjAFi',
   'content': "{'id': '2837622', 'name': 'Priya Patel', 'email': 'priya@candy.com', 'phone': '987-654-3210', 'username': 'priya123'}"}]}

### The Complete Chatbot

In [22]:
import re
def extract_reply(text):
    pattern = r'<reply>(.*?)</reply>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return None    
    
def simple_chat():
    system_prompt = """
    You are a customer support chat bot for an online retailer
    called Acme Co.Your job is to help users look up their account, 
    orders, and cancel orders.Be helpful and brief in your responses.
    You have access to a set of tools, but only use them when needed.  
    If you do not have enough information to use a tool correctly, 
    ask a user follow up questions to get the required inputs.
    Do not call any of the tools unless you have the required 
    data from a user. 

    In each conversational turn, you will begin by thinking about 
    your response. Once you're done, you will write a user-facing 
    response. It's important to place all user-facing conversational 
    responses in <reply></reply> XML tags to make them easy to parse.
    """
    user_message = input("\nUser: ")
    messages = [{"role": "user", "content": user_message}]
    while True:
        if user_message == "quit":
            break
        #If the last message is from the assistant, 
        # get another input from the user
        if messages[-1].get("role") == "assistant":
            user_message = input("\nUser: ")
            messages.append({"role": "user", "content": user_message})

        #Send a request to Claude
        response = client.messages.create(
            model=MODEL_NAME,
            system=system_prompt,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )
        # Update messages to include Claude's response
        messages.append(
            {"role": "assistant", "content": response.content}
        )

        #If Claude stops because it wants to use a tool:
        if response.stop_reason == "tool_use":
            #Naive approach assumes only 1 tool is called at a time
            tool_use = response.content[-1] 
            tool_name = tool_use.name
            tool_input = tool_use.input
            print(f"=====Claude wants to use the {tool_name} tool=====")


            #Actually run the underlying tool functionality on our db
            tool_result = process_tool_call(tool_name, tool_input)

            #Add our tool_result message:
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": str(tool_result),
                        }
                    ],
                },
            )
        else: 
            #If Claude does NOT want to use a tool, 
            #just print out the text reponse
            model_reply = extract_reply(response.content[0].text)
            print("\nAcme Co Support: " + f"{model_reply}" )

In [ ]:
# user input:
# hello!
# I want to check my order status
# I have my username which is priya123
# 
simple_chat()


User:  hello!



Acme Co Support: Hello! I'm the Acme Co. customer service bot. I can help you look up your account information, check your orders, or help with order cancellations. How can I assist you today?



User:  I want to check my order status



Acme Co Support: I'll be happy to help you check your order status. Could you please provide either:
1. Your specific order ID
or
2. Your email, phone number, or username so I can look up your orders?



User:  I have my username which is priya123


...

=====Claude wants to use the cancel_order tool=====

Acme Co Support: I've successfully cancelled your order #28164 for the Wireless Headphones. The cancellation has been processed and you should receive a confirmation email shortly. Is there anything else I can help you with?



User:  Please check the order status for my username which is priya123 


=====Claude wants to use the get_user tool=====
=====Claude wants to use the get_customer_orders tool=====

Acme Co Support: Here are your current orders:
1. Order #97531: 1 Bluetooth Speaker ($49.99) - Status: Shipped
2. Order #28164: 2 Wireless Headphones ($79.99) - Status: Cancelled

As you can see, your Bluetooth Speaker has been shipped, and your Wireless Headphones order has been successfully cancelled as requested. Can I help you with anything else?



User:  Thank you, let's just quit



Acme Co Support: You're welcome! Thanks for chatting with Acme Co. customer service. Have a great day! 👋
